In [ ]:
# Imports

import keras
import datetime as dt
import os
import numpy as np
import tensorflow as tf
from datetime import datetime
from os import listdir
from os.path import isfile, join

In [ ]:
# Create model

input_shape1 = (2, 7, 6)
input_shape2 = (4,)
input_shape3 = (4,)
array_input = keras.layers.Input(shape = (2, 7, 6), name = "array_input")

input1 = keras.layers.Input(shape=input_shape1, name="regression: board and lines")
input2 = keras.layers.Input(shape=input_shape2)
input3 = keras.layers.Input(shape=input_shape3)

flatten1 = keras.layers.Flatten()(input1)

concatenated = keras.layers.Concatenate()([flatten1, input2, input3])

# Dense layers for processing concatenated inputs
dense1 = keras.layers.Dense(1024, activation = "relu")(concatenated)
dense2 = keras.layers.Dense(512, activation = "relu")(dense1)
dense3 = keras.layers.Dense(256, activation = "relu")(dense2)

# Output layer
output = keras.layers.Dense(1, activation = "linear", name = "output")(dense3)

model = keras.models.Model(inputs=[input1, input2, input3], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()


In [ ]:
# Save model

model_name = "board-and-lines"

if not os.path.exists(f"models/{model_name}"):
    os.makedirs(f"models/{model_name}")
model.save(f"models/{model_name}/{dt.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")


In [ ]:
# Train model

ys = np.load("y.npy")
positions = np.load("positions.npy")
my_lines = np.load("my_lines.npy")
enemy_lines = np.load("enemy_lines.npy")

while True:
    model_version = sorted([f for f in listdir(f"models/{model_name}") if isfile(join(f"models/{model_name}", f))])[-1]
    print(len(ys))
    model = tf.keras.models.load_model(f"models/{model_name}/{model_version}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

    model.fit([positions, my_lines, enemy_lines], ys, epochs = 1, batch_size = 128, validation_split = 0.2)

    model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")
